<a href="https://colab.research.google.com/github/nayoung718/essa_python_1/blob/main/3_15_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%84%B8%EC%85%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___

수백만 개의 특성이 있는 훈련 세트를 가지고 있다면, 확률적 경사 하강법이나 미니배치 경사 하강법을 사용할 수 있다. 훈련 세트가 메모리 크기에 맞으면 배치 경사 하강법도 가능하다. 하지만 정규 방정식이나 SVD 방법은 계산 복잡도가 특성 개수에 따라 매우 빠르게 증가하기 때문에 사용할 수 없다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

에포크마다 검증 에러가 지속적으로 상승한다면 한 가지 가능성은 학습률이 너무 높아 알고리즘이 발산하는 것일 수 있다. 훈련 에러도 올라간다면 이 문제가 확실하므로, 학습률을 낮추어야 한다. 하지만 훈련 에러가 증가하지 않는다면, 모델이 훈련 세트에 과대적합 되어 있는 것이므로 훈련을 멈춰야 한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

훈련 에러와 검증 에러가 거의 비슷하고 매우 높다면, 모델이 훈련 세트에 과소적합되었을 가능성이 높다. 즉, 높은 편향을 가진 모델이기 때문에, 규제 하이퍼파라미터 α 를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1. 평범한 선형 회귀 대신 릿지 회귀: 규제가 있는 모델이 일반적으로 규제가 없는 모델보다 성능이 좋기 때문이다.

2. 릿대회귀대신 라쏘 회귀: 라쏘 회귀는 페널티를 사용하여, 가중치를 완전히 0으로 만드는 경향이 있다. 이는 가장 중요한 가중치를 제외하고는 모두 0이 되는 희소한 모델을 만든다. 또한 자동으로 특성 선택의 효과를 가지므로, 일부 특성만 실제로 유용할 것 같을 때 사용하면 좋다. 확신이 없다면 릿지 회귀를 사용해야 한다.

3. 라쏘 회귀 대신 엘라스틱넷: 라쏘가 어떤 경우에는 불규칙하게 행동하므로, 엘라스틱넷이 라쏘보다 일반적으로 선호된다. 그러나 추가적인 하이퍼 파라미터가 생긴다. 불규칙한 행동이 없는 라쏘를 원하면 엘라스틱 넷에 l1_ratio를 1에 가깝게 설정하면 좋다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris['data'][:,(2,3)]
y = iris['target']

In [3]:
X_with_bias = np.c_[np.ones([len(X),1]),X]

In [4]:
np.random.seed(1234)
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)    # 150을 무작위로 섞음

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]  # test_size 만큼 남겨둠
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

def to_one_hot(y):
    n_classes = y.max()+1    # 0,1,2 라 max=2 / +1 하면 classes 개수
    m = len(y)                     # 총 150개의 라벨들
    y_one_hot = np.zeros((m,n_classes))
    y_one_hot[np.arange(m),y] = 1   # index의 행중에 y값을 1로 치환
    return y_one_hot

# 라벨 전부를 onehot encoding하기
y_train_one_hot = to_one_hot(y_train)
y_valid_one_hot = to_one_hot(y_valid)
y_test_one_hot = to_one_hot(y_test)

In [5]:
# Softmax 함수 만들기

def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps,axis=1,keepdims=True)   # axis=1   ->  가장 안쪽의 [ ] 안의 성분의 합 / 각 exps들의 합
    return exps/exp_sums

# 입력과 출력의 갯수 정하기
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [6]:
eta = 0.01
n_iteration = 5001
m = len(X_train)
epsilon = 1e-7 # ε : 입실론     nan값을 피하기 위해 logPi에 추가.

Theta = np.random.randn(n_inputs,n_outputs)

for i in range(n_iteration):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    loss = -np.mean(np.sum(y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    error = Y_proba - y_train_one_hot
    if i % 500 == 0:
        print(i,loss)
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 3.1114915802681646
500 0.8785451855619314
1000 0.7103533106437264
1500 0.6136193727911318
2000 0.5518526140625929
2500 0.5086585463656746
3000 0.4762907516068719
3500 0.45076167129780975
4000 0.42984346695974646
4500 0.41220116725499784
5000 0.39698683289431363


In [7]:
# 검증 세트에 대한 정확도 확인
logits = X_valid.dot(Theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy = np.mean(y_predict == y_valid)
accuracy

0.9